In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Importing Dataset

In [2]:
df = pd.read_csv('../input/d/jpmiller/how-people-get-hurt/NEISS.TXT', sep='\t', encoding='latin-1')

In [3]:
df.head()

# Data Wrangling

- Uneccesary columns are dropped
    - **Hispanic**: Incorporated into the race column.
    - **Stratum**: Design Variable-Stratum unneeded.
    - **PSU**: Design Variable-Primary Sampling Unit (PSU) unneeded.
    - **Weight**: Statistical Weight for National Estimates unneeded.
    - **Other_race, other_diagnosis, other_diagnosis_2, body_part_2, diagnosis_2, product_2, product_3**: applies to multiple variables of the same type, but for simplicity the main variable is used only.


- Dropped rows with 'age' from 201-223 representing infants from 1 - 23 months.
- Dropped rows with 'age' = 0, 'sex' = 0, 'race' = 0, 'disposition' = 9, 'location' = 0 representing values not known.
- Dropped rows with N/A and NAN values

- Reset indexes of rows once all undesired rows are removed.

- Changed type of 'Treatment Date' from object to DateTime.

In [4]:
df = df.drop(columns=["Hispanic",
                       "Stratum", 
                       "PSU", 
                       "Weight",
                       "Other_Race",
                       "Other_Diagnosis", 
                       "Other_Diagnosis_2", 
                       "Body_Part_2",
                       "Diagnosis_2", 
                       "Product_2", 
                       "Product_3"])
df = df.rename(columns={"Product_1":"Product"})
df = df.drop(df[df.Age > 120].index)
df = df.drop(df[df.Age == 0].index)
df = df.drop(df[df.Sex == 0].index)
df = df.drop(df[df.Race == 0].index)
df = df.drop(df[df.Disposition == 9].index)
df = df.drop(df[df.Location == 0].index)
df = df.dropna()

In [5]:
df = df.reset_index(drop=True)
df.head()

In [7]:
df.dtypes

In [12]:
df["Treatment_Date"] = pd.to_datetime(df["Treatment_Date"], format='%m/%d/%Y')
df.dtypes

# Data Transformation

To get a better image of the data, numerical variables that should be categorical variables are converted back to strings using the Data Dictionary spreadsheet given:
- Sex
- Race
- Body part
- Diagnosis
- Disposition
- Location
- Fire involvement
- Alcohol involvement
- Drug involvement.

For the product string conversion, the NEISS_FMT.txt was used instead:
- Rows that did not contain product information was dropped
- Rows were sliced and converted to two columns:
    - First column contains product number
    - Second column contains product description
- The two columns were then converted into a dictionary:
    - Key: product number
    - Value: product description
- Each product number in the main dataset was replaced then by the corresponding value in the dictionary

In [14]:
#Create dictionary of values for numerical translation
sex_dict = {1 : 'Male',
            2 : 'Female'}

race_dict = {1 : 'White',
            2 : 'Black/African American',
            3 : 'Other',
            4 : 'Asian',
            5 : 'American Indian/Alaska Native',
            6 : 'Native Hawaiian/Pacific Islander'}

body_part_dict = {0.0 : 'Internal',
            30 : 'Shoulder',
            31 : 'Upper Trunk',
            32 : 'Elbow',
            33 : 'Lower Arm',
            34 : 'Wrist',
            35 : 'Knee',
            36 : 'Lower Leg',
            37 : 'Ankle',
            38 : 'Pubic Region',
            75 : 'Head',
            76 : 'Face',
            77 : 'Eyeball',
            79 : 'Lower Trunk',
            80 : 'Upper Arm',
            81 : 'Upper Leg',
            82 : 'Hand',
            83 : 'Foot',
            84 : '25-50% of Body',
            85 : 'All Parts Body',
            87 : 'Not Stated/Unk',
            88 : 'Mouth',
            89 : 'Neck',
            92 : 'Finger',
            93 : 'Toe',
            94 : 'Ear'}

diagnosis_dict = {41 : 'Ingestion',
            42 : 'Aspiration',
            46 : 'Burns, Electrical',
            47 : 'Burns, Not Specified',
            48 : 'Burns, Scald',
            49 : 'Burns, Chemical',
            50 : 'Amputaion',
            51 : 'Burns, Thermal',
            52 : 'Concussions',
            53 : 'Contusions, Abrasions',
            54 : 'Crushing',
            55 : 'Dislocation',
            56 : 'Foreign Body',
            57 : 'Fracture',
            58 : 'Hematoma',
            59 : 'Laceration',
            60 : 'Dental Injury',
            61 : 'Nerve Damage',
            62 : 'Internal Organ Injury',
            63 : 'Puncture',
            64 : 'Strain, Sprain',
            65 : 'Anoxia',
            66 : 'Hemorrhage',
            67 : 'Electric Shock',
            68 : 'Poisoning',
            69 : 'Submersion',
            71 : 'Other/Not Stated',
            72 : 'Avulsion',
            73 : 'Burns, Radiation',
            74 : 'Dermatitis, Conjunctivitis'}

disp_dict = {1 : 'Treated/Examined and Released',
            2 : 'Treated and Transferred',
            4 : 'Treated and Admitted/Hospitalized',
            5 : 'Held for Observation',
            6 : 'Left Without Being Seen',
            8 : 'Fatality, Incl. DOA, Died in ER',
            9 : 'Unknown, Not Stated'}

loc_dict = {0 : 'Not Recorded',
            1 : 'Home',
            2 : 'Farm/Ranch',
            4 : 'Street or Highway',
            5 : 'Other Public Property',
            6 : 'Mobile/Manufactured Home',
            7 : 'Industrial',
            8 : 'School/Daycare',
            9 : 'Place of Recreation or Sports'}

fire_dict = {0 : 'No Fire Involved or Fire Involvement Not Recorded',
            1 : 'Fire Involved and Fire Department Attended',
            2 : 'Fire Involved and Fire Department Did Not Attend',
            3 : 'Fire Involved and Unknown Fire Department Attendance'}

alcohol_dict = {0 : 'No/No Information',
            1 : 'Yes'}

drug_dict = {0 : 'No/No information',
            1 : 'Yes'}

In [15]:
df_labeled = df
df_labeled = df_labeled.replace({'Sex' : sex_dict})
df_labeled = df_labeled.replace({'Race' : race_dict})
df_labeled = df_labeled.replace({'Body_Part' : body_part_dict})
df_labeled = df_labeled.replace({'Diagnosis' : diagnosis_dict})
df_labeled = df_labeled.replace({'Disposition' : disp_dict})
df_labeled = df_labeled.replace({'Location' : loc_dict})
df_labeled = df_labeled.replace({'Fire_Involvement' : fire_dict})
df_labeled = df_labeled.replace({'Alcohol' : alcohol_dict})
df_labeled = df_labeled.replace({'Drug' : drug_dict})
df_labeled.head()

In [17]:
df_format = pd.read_csv('../input/d/jpmiller/how-people-get-hurt/NEISS_FMT.TXT', sep='\t', encoding='latin-1')

In [18]:
df_format = df_format.drop(df_format[df_format['Format name'] != 'PROD'].index)
df_format = df_format.drop(columns=['Starting value for format', 'Ending value for format'])

In [19]:
df_format['Format name'] = df_format['Format value label'].str.slice(0,4)
df_format['Format value label'] = df_format['Format value label'].str.slice(6,)

In [20]:
df_format = df_format.rename(columns={'Format name' : 'Number', 'Format value label' : 'Name'})

In [21]:
df_format.head()

In [22]:
prod_dict = dict(zip(df_format.Number, df_format.Name))

In [23]:
prod_dict = dict([float(a), x] for a, x in prod_dict.items())

In [26]:
df_labeled = df_labeled.replace({'Product' : prod_dict})
df_labeled.head()

In [27]:
df_labeled.to_csv("NEISS Dataset.csv")